In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train=pd.read_csv("Training_set.csv")

In [3]:
train=train.sample(20000)

In [4]:
test=pd.read_csv("Test_set.csv")

In [5]:
def get_job_characteristics(df): 
    job_type=[]
    job_title=[]
    employer_title=[]
    employer_sector_title=[]
    job_sector_title=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        types=data["job_type"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        job_type.append(types)
        job_title.append(title)
        employer_title.append(em_title)
        employer_sector_title.append(em_sec_title)
        job_sector_title.append(job_sec_title)
    d={}
    #d["job_title"]=job_title
    #d["employer_title"]=employer_title
    d["employer_sector_title"]=employer_sector_title
    d["job_sector_title"]=job_sector_title
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(job_type)
    result = pd.concat([data, b], axis=1, sort=False)
    return result

In [6]:
def get_data_for_user(user):
    jobtitles=list(user["job_title"].values)
    emtitles=list(user["employer_title"].values)
    jobsectitles=list(user["job_sector_title"].values)
    emsectitles=list(user["employer_sector_title"].values)
    a=""
    for k in range(len(jobtitles)):
        a+=jobtitles[k]+" "
        a+=emtitles[k]+" "
        a+=jobsectitles[k]+" "
        #a+=emsectitles[k]+" "
    return a

In [7]:
def get_user_characteristics(df):
    corpus=[]
    gender=[]
    ethnicity=[]
    school_type=[]
    right_to_work_uk=[]
    university_name=[]
    university_type=[]
    course_includes_industrial_placement=[]
    degree_subject_name=[]
    user_preference=[]
    job_types=list(set(df["job_type"].values))
    em_sectors=list(set(df["employer_sector_title"].values))
    job_sectors=list(set(df["job_sector_title"].values))
    count_job_types=[[] for i in range(len(job_types))]
    count_job_sectors=[[] for i in range(len(job_sectors))]
    count_em_sectors=[[] for i in range(len(em_sectors))]
    nb_em_sector=[]
    nb_job_sector=[]
    job_type=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        user=data["user_id"]
        user_data=df[df["user_id"]==user]
        user_job_type=dict(Counter(list(user_data["job_type"].values)))
        user_job_sectors=dict(Counter(list(user_data["job_sector_title"].values)))
        user_em_sectors=dict(Counter(list(user_data["employer_sector_title"].values)))
        user_em_sector=len(list(set(list(user_data["employer_sector_title"].values))))
        user_job_sector=len(list(set(list(user_data["job_sector_title"].values))))
        for i in range(len(count_job_types)):
            try:
                count_job_types[i].append(user_job_type[i])
            except:
                count_job_types[i].append(0)
        for i in range(len(count_job_sectors)):
            try:
                count_job_sectors[i].append(user_job_sectors[i])
            except:
                count_job_sectors[i].append(0)
        for i in range(len(count_em_sectors)):
            try:
                count_em_sectors[i].append(user_em_sectors[i])
            except:
                count_em_sectors[i].append(0)
        sex=data["gender"]
        jobtype=data["job_type"]
        eth=data["ethnicity"]
        school=data["school_type"]
        placement=data["course_includes_industrial_placement"]
        right=data["right_to_work_uk"]
        uni=data["university_name"]
        uni_type=data["university_type"]
        subname=data["degree_subject_name"]
        pref=data["user_preference"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        if sex=="F":
            sex="Female"
        elif sex=="M":
            sex="Male"
        if placement==True:
            placement=1
        elif placement==False:
            placement=-1
        else:
            placement=0
        if right==True:
            right=1
        elif right==False:
            right=-1
        else:
            right=0
        if type(pref) is float:
            pref=""
        if type(subname) is float:
            subname=""
        nb_em_sector.append(user_em_sector)
        nb_job_sector.append(user_job_sector)
        gender.append(sex)
        ethnicity.append(eth)
        school_type.append(school)
        course_includes_industrial_placement.append(placement)
        right_to_work_uk.append(right)
        university_type.append(uni_type)
        #degree_subject_name.append(subname)
        #user_preference.append(pref)
        job_type.append(jobtype)
        corpus.append(pref+subname+" "+get_data_for_user(user_data))
    d={}
    for i in range(len(count_job_sectors)):
        d['count_job_sectors_'+str(i)]=count_job_sectors[i]
    for i in range(len(count_em_sectors)):
        d['count_em_sectors_'+str(i)]=count_em_sectors[i]
    for i in range(len(count_job_types)):
        d['count_job_types_'+str(i)]=count_job_types[i]
    d["gender"]=gender
    d["job_type"]=job_type
    d["nb_em_sector"]=nb_em_sector
    d["nb_job_sector"]=nb_job_sector
    d["right_to_work_uk"]=right_to_work_uk
    d["ethnicity"]=ethnicity
    #d["degree_subject_name"]=degree_subject_name
    d["school_type"]=school_type
    #d["user_preference"]=user_preference
    d["course_includes_industrial_placement"]=course_includes_industrial_placement
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(university_type)
    result = pd.concat([data, b], axis=1, sort=False)
    vectorizer = CountVectorizer(max_df=1.0, min_df=3)
    X = vectorizer.fit_transform(corpus)
    col=vectorizer.get_feature_names()
    donnees=X.toarray()
    print(X.shape)
    donnee=pd.DataFrame(donnees, columns=col)
    final=pd.concat([result, donnee], axis=1)
    return final

In [8]:
#user_characteristics=get_user_characteristics(train)

In [9]:
#user_characteristics.shape

In [10]:
#job_characteristics=get_job_characteristics(train)

In [11]:
#job_characteristics.shape

In [12]:
def get_user_info(df):
    user_info=[]
    x=train[["user_id","job_title","employer_title","job_sector_title","job_description",
             "employer_sector_title","employer_description"]]
    n=df.shape[0]
    for i in range(n):
        if i%5000==0:
            print("progress: ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        y=x[x["user_id"]==user_id]
        job_titles=y["job_title"].values
        job_sector_title=y["job_sector_title"].values
        description=y["job_description"].values
        em_titles=y["employer_title"].values
        em_sector_title=y["employer_sector_title"].values
        em_description=y["employer_description"].values
        university_name=data["university_name"]
        degree_subject_name=data["degree_subject_name"]
        user_preference=data["user_preference"]
        ethnicity=data["ethnicity"]
        school_type=data["school_type"]
        if type(user_preference) is float:
            user_preference=""
        if type(university_name) is float:
            university_name=""
        if type(degree_subject_name) is float:
            degree_subject_name=""
        if type(ethnicity) is float:
            ethnicity=""
        if type(school_type) is float:
            school_type=""
        user_preference=user_preference+" "
        user_preference=user_preference *3
        degree_subject_name=degree_subject_name+" "
        degree_subject_name=degree_subject_name *3
        a=""
        a+=university_name+" "
        a+=degree_subject_name+" "
        a+=user_preference+" "
        a+=ethnicity+" "
        a+=school_type+" "
        for k in range(len(job_titles)):
            a+=job_titles[k]+" "
            a+=job_sector_title[k]+" "
            a+=description[k]+" "
            a+=em_titles[k]+" "
            a+=em_sector_title[k]+" "
            a+=em_description[k]+" "
        a=a.replace("Graduate","").replace("Programme","").replace("  "," ").replace("&","")
        user_info.append(a)
    return user_info

In [13]:
train_user_info=get_user_info(train)

progress:  0.0
progress:  0.25
progress:  0.5
progress:  0.75


In [14]:
def clean_user(x):
    n=len(x)
    res=[] 
    for i,content in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        content=content.replace(",","").replace("-"," ").replace("  "," ").replace("&eacute;","e")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        content=content.replace("&ldquo;","").replace("ldquo;","").replace("&rdquo;","").replace("rdquo;","")
        try:
            content=''.join([i for i in content if not i.isdigit()])
        except:
            pass
        content=content.split(" ")
        y=[]
        for i in content:
            if len(i)>1:
                y.append(i.lower())
        res.append(y)
    return res

In [15]:
clean_train_user_info=clean_user(train_user_info)

progress :  0.0
progress :  0.25
progress :  0.5
progress :  0.75


In [16]:
def get_job_info(df):
    job_info=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        title=data["job_title"]
        job_sector_title=data["job_sector_title"]
        description=data["job_description"]
        em_title=data["employer_title"]
        em_sector_title=data["employer_sector_title"]
        em_description=data["employer_description"]
        a=""
        a+=title+" "
        a+=job_sector_title+" "
        a+=description+" "
        a+=em_title+" "
        a+=em_sector_title+" "
        a+=em_description
        job_info.append(a)
    return job_info

In [17]:
train_job_info=get_job_info(train)

In [18]:
def clean(x):
    res=[] 
    for content in x:
        content=content.replace(",","").replace("-"," ").replace("  "," ").replace("&eacute;","e")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        content=content.replace("&ldquo;","").replace("ldquo;","").replace("&rdquo;","").replace("rdquo;","")
        try:
            content=''.join([i for i in content if not i.isdigit()])
        except:
            pass
        content=content.split(" ")
        y=[]
        for i in content:
            if len(i)>1:
                y.append(i.lower())
        res.append(y)
    return res

In [19]:
cleaned_train_job_info=clean(train_job_info)

In [20]:
def remove_stopwords(x):
    res=[]
    with open('stopwords.txt') as stopfile:
        stopwords = stopfile.read()
        stop = stopwords.split()
    for i,job in enumerate(x):
        a=[]
        for word in job:
            if word not in stop:
                a.append(word)
        res.append(a)
    return res

In [21]:
train_clean=remove_stopwords(cleaned_train_job_info)

In [22]:
train_user_clean=remove_stopwords(clean_train_user_info)

In [23]:
def get_occurence(x):
    res=[]
    for job in x:
        res.append(dict(Counter(job)))
    return res

In [24]:
occurences=get_occurence(train_clean)

In [25]:
train_user_occurences=get_occurence(train_user_clean)

In [26]:
def get_vector(occurences, jobs, vector_size):
    vector=[]
    model = Word2Vec.load('model.bin')
    nb=len(jobs)
    for i,job in enumerate(jobs):
        if i%10000==0:
            print("progress : ",i/nb )
        res=np.zeros(vector_size,)
        n=sum(list(occurences[i].values()))
        job_unique=list(set(job))
        for word in job_unique:
            try:
                a=model[word]
            except:
                print(word)
                a=None
            if a is not None:
                count=occurences[i][word]
                w=count/n
                res+=w*a
        vector.append(res)
    return vector

In [ ]:
X=get_vector(occurences, train_clean, 200)

In [ ]:
X_user=get_vector(train_user_occurences, train_user_clean, 200)

In [29]:
X_train=np.array(X)

In [30]:
X_train.shape

(20000, 200)

In [31]:
X_train_user=np.array(X_user)

In [32]:
X_train_user.shape

(20000, 200)

In [33]:
#X_train_job_final=np.concatenate((X_train, job_characteristics.values), axis=1)

In [34]:
#X_train_job_final.shape

In [35]:
#X_train_user_final=np.concatenate((X_train_user, user_characteristics.values), axis=1)

In [36]:
#X_train_user_final.shape

In [37]:
#pca = PCA(n_components=2)
#result_user = pca.fit_transform(X_train_user_final)

In [38]:
#print(pca.explained_variance_ratio_)

In [39]:
#pca = PCA(n_components=2)
#result = pca.fit_transform(X_train_job_final)

In [40]:
#print(pca.explained_variance_ratio_)

In [41]:
#users=list(train["user_id"].values)
#users=[str(i) for i in users]

In [42]:
def get_job_info_plot(df):
    job_info=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        title=data["job_title"]
        em_title=data["employer_title"]
        a=""
        a+=title+" "
        a+=em_title+" "
        job_info.append(a)
    return job_info

In [43]:
#infos=get_job_info_plot(train)

In [44]:
"""k=200
plt.figure(figsize=(20,20))
plt.scatter(result_user[:k, 0], result_user[:k, 1])
for i, word in enumerate(users[:k]):
    plt.annotate(word, xy=(result_user[i, 0], result_user[i, 1]))
plt.show()"""

'k=200\nplt.figure(figsize=(20,20))\nplt.scatter(result_user[:k, 0], result_user[:k, 1])\nfor i, word in enumerate(users[:k]):\n    plt.annotate(word, xy=(result_user[i, 0], result_user[i, 1]))\nplt.show()'

In [50]:
train.iloc[10][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']]

job_title                Undergraduate Summer Placement 2018 ? Material...
employer_title                                              Balfour Beatty
employer_sector_title          Technology: Consulting & Project Management
job_sector_title                                               Engineering
gender                                                              Female
ethnicity                                              Asian/Asian British
university_name                                       Middlesex University
degree_subject_name                                 Engineering Management
user_preference          Investment Banking Insurance & Risk Management...
Name: 61081, dtype: object

In [51]:
def get_similar_member(df, X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(df.iloc[most_similar_doc_id[0]]["user_id"])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            user=df.iloc[most_similar_doc_id[i]]["user_id"]
            if user not in res:
                res.append(user)
                beta.append(cos[most_similar_doc_id[i]])
    print(" user id : ", df.iloc[doc_id]["user_id"])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [52]:
get_similar_member(train, X_train_user, 10, 5)

 user id :  182893
most similar doc :  [185282, 187746, 188123, 176102, 154347]
most similar values :  [0.9613004905310516, 0.960923090176145, 0.9558614880907775, 0.9369530938940134, 0.8387473795525148]
not similar doc :  [181654, 119699, 192324, 185756, 184009]
not similar values :  [-0.09057017489310004, -0.019357563765597885, 0.140486744238113, 0.14540056942671373, 0.1501322723557333]


# user

In [53]:
train[train["user_id"]==182893][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(10)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
61081,Undergraduate Summer Placement 2018 ? Material...,Balfour Beatty,Technology: Consulting & Project Management,Engineering,Female,Asian/Asian British,Middlesex University,Engineering Management,Investment Banking Insurance & Risk Management...


# similar users

In [54]:
train[train["user_id"]==185282][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail()

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
75135,Undergraduate Summer Placement 2018 ? Material...,Balfour Beatty,Technology: Consulting & Project Management,Engineering,Female,White/White British,University of Leeds,Natural Sciences,Investment Banking Energy & Infrastructure Eng...


In [55]:
train[train["user_id"]==176102][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
33978,Undergraduate Summer Placement 2018 ? Material...,Balfour Beatty,Technology: Consulting & Project Management,Engineering,Female,White/White British,Imperial College London,Biochemistry,NaN


# not similar users

In [56]:
train[train["user_id"]==181654][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
54126,Herbert Smith Freehills Event Dates 2017-2018,Herbert Smith Freehills,Commercial Law,Commercial Law,Male,Asian/Asian British,London School of Economics and Political Scien...,Economics,NaN


In [57]:
train[train["user_id"]==119699][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
74175,Brand Ambassadors,Savse,"Consumer, FMCG & Retail","Consumer, FMCG & Retail",Female,Asian/Asian British,Queen Mary University of London,English,NaN


In [70]:
train[train["user_id"]==98253][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
125035,Campus Ambassadors,Reed Smith,Commercial Law,Commercial Law,Female,NaN,Durham University,History and Spanish,Commercial Law


In [58]:
def get_similar_doc(X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(most_similar_doc_id[0])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            res.append(most_similar_doc_id[i])
            beta.append(cos[most_similar_doc_id[i]])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [59]:
def clean_similarity(x):
    res=[] 
    for content in x:
        content=content.replace(",","").replace("-"," ").replace("  "," ")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        res.append(content)
    return res

In [62]:
get_similar_doc(X_train, 1235, 5)

most similar doc :  [5794, 10364, 5862, 16232, 18946]
most similar values :  [0.9231850621219728, 0.8811004908730489, 0.8723324770599936, 0.8575069173319624, 0.8379069345216089]
not similar doc :  [17229, 17972, 2745, 7386, 16956]
not similar values :  [-0.012092525520003384, 0.027450440621638334, 0.09623801782329758, 0.10239768281472389, 0.10516588989399617]


In [61]:
data_similarity=clean_similarity(train_job_info)

# job

In [63]:
data_similarity[1235]

'Investment Consulting Internship Financial Services Introduction to the Stream As the world leader in Investment Consulting Aon has been advising employers and governments on their pension schemes and other assets for over 60 years Today we provide a wide range of clients with expertise that extends from setting their investment strategies and the day to day management of their investment portfolios through to corporate transactions and transfers of assets from one Investment Manager to another  Aon advises clients on a number of strategies that have taken advantage of recent financial market conditions enabling their employees to enjoy retirement  Summer Internship Locations Bristol Birmingham Edinburgh Farnborough Leeds London St Albans 12 Month Industrial Placement Locations Bristol and Birmingham The role As a summer intern or industrial placement student you will gain a deep understanding of investment consulting and the potential career path ahead of you You start the internship

# Similar jobs

In [64]:
data_similarity[5794]

'Investment Consulting Financial Services Introduction to the Stream As the world leader in Investment Consulting Aon has been advising employers and governments on their pension schemes and other assets for over 60 years Today we provide a wide range of clients with expertise that extends from setting their investment strategies and the day to day management of their investment portfolios through to corporate transactions and transfers of assets from one Investment Manager to another  Aon advises clients on a number of strategies that have taken advantage of recent financial market conditions enabling their employees to enjoy retirement  Locations Birmingham Bristol Edinburgh Farnborough Leeds Manchester St Albans London The role As a Graduate Investment Analyst you will be a key member of our dedicated team playing a vital role in meeting the expectations of our global clients through the delivery of high quality servicing and advice This will entail providing investment advice cover

# not similar jobs

In [65]:
data_similarity[17229]

'Herbert Smith Freehills Event Dates 2017 2018 Commercial Law There are lots of opportunities to meet with key decision makers at top commercial law firm Herbert Smith Freehills  Open Days The IP Open day applications will be open from 1 October 2017 10 November 2017 and will take place on 8 December 2017  The Disputes Open Day applications will be open from 18 September 16 October and will take place on 10 November 2017  Click here to apply  Date Tuesday 3rd October 2017 Event Bristol Inside the Headlines Presentation Venue Mercure Bristol Grand Hotel Broad Street Time 6 9pm   Date Wednesday 4th October 2017  Event TCD Inside the Headlines Presentation  Venue The Spencer Hotel Excise Walk IFSC Dublin 1 Ireland  Time 6 9pm   Date Thursday 5th October 2017 Event QUB Inside the Headlines Presentation Venue Moot Court Room Queen University Belfast School of Law 27 30 University Square Belfast BT7 1NN Time 6 9pm   Date Monday 9th October 2017 Event Warwick Inside the Headlines Presentation

In [43]:
X_train.shape

(125136, 200)

In [44]:
job_characteristics.shape

(125136, 6)